#Group By

In [0]:
# Read data into a DataFrame
data = [("Alice", "Sales", 4000),
        ("Bob", "HR", 5000),
        ("Carol", "Sales", 6000),
        ("Dave", "IT", 5500),
        ("Eve", "HR", 6000)]

columns = ["Name", "Department", "Salary"]
df = spark.createDataFrame(data,columns)

In [0]:
df.show(truncate=False)

+-----+----------+------+
|Name |Department|Salary|
+-----+----------+------+
|Alice|Sales     |4000  |
|Bob  |HR        |5000  |
|Carol|Sales     |6000  |
|Dave |IT        |5500  |
|Eve  |HR        |6000  |
+-----+----------+------+



In [0]:
gr = df.groupBy("Department").sum('Salary')
gr.show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      10000|
|        HR|      11000|
|        IT|       5500|
+----------+-----------+



In [0]:
grouped = df.groupBy("Department").sum('Salary')
grouped.show()




+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      10000|
|        HR|      11000|
|        IT|       5500|
+----------+-----------+



In [0]:
from pyspark.sql.functions import *

In [0]:
df_grouped = df.groupBy("Department").sum('Salary')
df_grouped.show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|      10000|
|        HR|      11000|
|        IT|       5500|
+----------+-----------+



In [0]:
df_grouped.count()

3

In [0]:
df_count.count()

3

In [0]:
df.groupBy("Department").agg(sum(col("Salary")),avg(col("Salary")),count(col("Salary"))).show()

+----------+-----------+-----------+-------------+
|Department|sum(Salary)|avg(Salary)|count(Salary)|
+----------+-----------+-----------+-------------+
|     Sales|      10000|     5000.0|            2|
|        HR|      11000|     5500.0|            2|
|        IT|       5500|     5500.0|            1|
+----------+-----------+-----------+-------------+



In [0]:
df.count()

5

##Pivot

In [0]:
data = [("Alice", "Math", 95),
        ("Alice", "Science", 88),
        ("Alice", "Moral Science", 88),
        ("Bob", "Math", 92),
        ("Bob", "Science", 90),
        ("Carol", "Math", 85),
        ("Carol", "Science", 78)]

columns = ["Name", "Subject", "Score"]
df = spark.createDataFrame(data, columns)
df.show()


+-----+-------------+-----+
| Name|      Subject|Score|
+-----+-------------+-----+
|Alice|         Math|   95|
|Alice|      Science|   88|
|Alice|Moral Science|   88|
|  Bob|         Math|   92|
|  Bob|      Science|   90|
|Carol|         Math|   85|
|Carol|      Science|   78|
+-----+-------------+-----+



In [0]:
pivot

In [0]:
pivoted_df = df.groupBy("Name").pivot("Subject").agg({"Score": "first"})

In [0]:
data = [("Alice", "Math", 95),
        ("Alice", "Science", 88),
        ("Alice", "Moral Science", 88),
        ("Carol", "Math", 85),
        ("Carol", "Science", 78),
        ("Bob", "Math", 92),
        ("Bob", "Science", 90)]

columns = ["Name", "Subject", "Score"]
df = spark.createDataFrame(data, columns)
df.show()

pivoted_df = df.groupBy("Name").pivot("Subject").agg({"Score": "last"})
pivoted_df.show()


+-----+-------------+-----+
| Name|      Subject|Score|
+-----+-------------+-----+
|Alice|         Math|   95|
|Alice|      Science|   88|
|Alice|Moral Science|   88|
|Carol|         Math|   85|
|Carol|      Science|   78|
|  Bob|         Math|   92|
|  Bob|      Science|   90|
+-----+-------------+-----+

+-----+----+-------------+-------+
| Name|Math|Moral Science|Science|
+-----+----+-------------+-------+
|Carol|  85|         NULL|     78|
|  Bob|  92|         NULL|     90|
|Alice|  95|           88|     88|
+-----+----+-------------+-------+



In [0]:
pivoted_df.show(truncate=False)

+-----+----+-------------+-------+
|Name |Math|Moral Science|Science|
+-----+----+-------------+-------+
|Carol|85  |NULL         |78     |
|Bob  |92  |NULL         |90     |
|Alice|95  |88           |88     |
+-----+----+-------------+-------+



In [0]:
pivoted_df.unpivot("Name", ["Math", "Moral Science","Science"], "Subjects", "Scores").show()

+-----+-------------+------+
| Name|     Subjects|Scores|
+-----+-------------+------+
|Carol|         Math|    85|
|Carol|Moral Science|  NULL|
|Carol|      Science|    78|
|  Bob|         Math|    92|
|  Bob|Moral Science|  NULL|
|  Bob|      Science|    90|
|Alice|         Math|    95|
|Alice|Moral Science|    88|
|Alice|      Science|    88|
+-----+-------------+------+



##Unpivot

In [0]:
df = spark.createDataFrame(
    [(1, 11, 1.1), (2, 12, 1.2)],
    ["id", "int", "double"],
)

df.show()


+---+---+------+
| id|int|double|
+---+---+------+
|  1| 11|   1.1|
|  2| 12|   1.2|
+---+---+------+



In [0]:

df.unpivot("id", ["int", "double"], "var", "val").show()


+---+------+----+
| id|   var| val|
+---+------+----+
|  1|   int|11.0|
|  1|double| 1.1|
|  2|   int|12.0|
|  2|double| 1.2|
+---+------+----+



#Window Functions

In [0]:
# Create a sample DataFrame
data = [("Sales", "Alice", 5000),
        ("Sales", "Bob", 6000),
        ("IT", "Carol", 7000),
        ("IT", "Dave", 8000),
        ("IT", "Eve", 9000)]

columns = ["department", "employee", "salary"]
df = spark.createDataFrame(data, columns)


In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import rank


In [0]:
window_spec = Window.partitionBy("department").orderBy(df["salary"])

In [0]:
ranked_df = df.withColumn("rank", rank().over(Window.partitionBy("department").orderBy(df["salary"])))

In [0]:
ranked_df.show(truncate=False)

+----------+--------+------+----+
|department|employee|salary|rank|
+----------+--------+------+----+
|IT        |Carol   |7000  |1   |
|IT        |Dave    |8000  |2   |
|IT        |Eve     |9000  |3   |
|Sales     |Alice   |5000  |1   |
|Sales     |Bob     |6000  |2   |
+----------+--------+------+----+



In [0]:
from pyspark.sql.functions import avg

window_spec = Window.partitionBy("department")

avg_salary_df = df.withColumn("avg_salary", avg(df["salary"]).over(window_spec))
avg_salary_df.show()



+----------+--------+------+----------+
|department|employee|salary|avg_salary|
+----------+--------+------+----------+
|        IT|   Carol|  7000|    8000.0|
|        IT|    Dave|  8000|    8000.0|
|        IT|     Eve|  9000|    8000.0|
|     Sales|   Alice|  5000|    5500.0|
|     Sales|     Bob|  6000|    5500.0|
+----------+--------+------+----------+



# Compute a rolling sum of salaries within each department for a specific range of rows.

In [0]:
from pyspark.sql.functions import sum

window_spec = Window.partitionBy("department").orderBy(df["salary"]).rowsBetween(-1, 1)

rolling_sum_df = df.withColumn("rolling_sum", sum(df["salary"]).over(window_spec))
rolling_sum_df.show()


+----------+--------+------+-----------+
|department|employee|salary|rolling_sum|
+----------+--------+------+-----------+
|        IT|   Carol|  7000|      15000|
|        IT|    Dave|  8000|      24000|
|        IT|     Eve|  9000|      17000|
|     Sales|   Alice|  5000|      11000|
|     Sales|     Bob|  6000|      11000|
+----------+--------+------+-----------+

